In [ ]:
data_store = {
    "users": [],   
    "tasks": [],   
    "logs": []     
}

class Student:
    def __init__(self, uid, name, major="Cyber Security"):
        self.uid = uid
        self.name = name
        self.major = major
        self.level = 1              
        self.score = 0              
        self.history = []           
        self.completed_tasks = []   

    def add_score(self, value):
        self.score = self.score + value
        self.history.append(value)

        if self.score >= 100 and self.level < 2:
            self.level = 2
        if self.score >= 250 and self.level < 3:
            self.level = 3

    def summary(self):
        total = 0
        count = 0
        i = 0
        while i < len(self.history):
            total = total + self.history[i]
            count = count + 1
            i = i + 1
        avg = 0
        if count > 0:
            avg = total / count
        return {
            "uid": self.uid,
            "name": self.name,
            "major": self.major,
            "level": self.level,
            "score": self.score,
            "task_count": count,
            "average": avg,
            "completed_tasks": self.completed_tasks
        }


class SecurityTask:
    def __init__(self, tid, title, steps, difficulty=1, category="general"):
        self.tid = tid
        self.title = title
        self.steps = steps
        self.difficulty = difficulty  
        self.category = category      
        self.completed_by = []         

    def base_score(self):
        step_count = len(self.steps)
        return self.difficulty * step_count * 5

    def mark_completed(self, uid):
        if uid not in self.completed_by:
            self.completed_by.append(uid)

class CyberLab:
    def __init__(self, store):
        self.store = store

    def add_user(self, uid, name):
        existing = self.find_user(uid)

        if existing is not None:
            return existing
            
        new_student = Student(uid, name)
        self.store["users"].append(new_student)
        self.add_log("NEW_STUDENT uid=" + str(uid) + " name=" + name)
        return new_student

    def find_user(self, uid):
        i = 0
        users = self.store["users"]
        while i < len(users):
            if users[i].uid == uid:
                return users[i]
            i = i + 1
        return None

    def update_score(self, uid, value):
        user = self.find_user(uid)
        if user is None:
            return False
        user.add_score(value)
        self.add_log("UPDATE_SCORE uid=" + str(uid) + " delta=" + str(value))
        return True

    def search_users_by_letter(self, letter):
        result = []
        users = self.store["users"]
        i = 0
        while i < len(users):
            u = users[i]
            if letter in u.name:
                result.append(u)
            i = i + 1
        return result

    def sort_users_by_score(self):
        users = self.store["users"]
        n = len(users)
        i = 0
        while i < n:
            j = 0
            while j < n - i - 1:
                if users[j].score < users[j + 1].score:
                    tmp = users[j]
                    users[j] = users[j + 1]
                    users[j + 1] = tmp
                j = j + 1
            i = i + 1
        return users

    def summarize_user(self, uid):
        user = self.find_user(uid)
        if user is None:
            return None
        return user.summary()

    def create_task(self, tid, title, steps, difficulty=1, category="general"):
        existing = self.get_task(tid)
        if existing is not None:
            return existing
        t = SecurityTask(tid, title, steps, difficulty, category)
        self.store["tasks"].append(t)
        self.add_log("NEW_TASK tid=" + str(tid) + " title=" + title)
        return t

    def get_task(self, tid):
        tasks = self.store["tasks"]
        i = 0
        while i < len(tasks):
            if tasks[i].tid == tid:
                return tasks[i]
            i = i + 1
        return None

    def complete_task(self, uid, tid):
        user = self.find_user(uid)
        task = self.get_task(tid)
        if user is None or task is None:
            return False

        if uid in task.completed_by:
            return False

        gained = task.base_score()
        self.update_score(uid, gained)
        task.mark_completed(uid)
        
        if tid not in user.completed_tasks:
            user.completed_tasks.append(tid)
        self.add_log("COMPLETE_TASK uid=" + str(uid) + " tid=" + str(tid) +
                     " gained=" + str(gained))
        return True

    def verify_step(self, step_text, requirement):
        matched = 0
        i = 0
        while i < len(requirement):
            c = requirement[i]
            if c in step_text:
                matched = matched + 1
            i = i + 1
        return matched == len(requirement)

    def process_steps(self, uid, tid, provided_steps):
        task = self.get_task(tid)
        if task is None:
            self.add_log("PROCESS_STEPS_FAILED missing_task tid=" + str(tid))
            return False
        required = task.steps
        if len(required) != len(provided_steps):
            self.add_log("PROCESS_STEPS_FAILED wrong_step_count tid=" + str(tid))
            return False
        i = 0
        while i < len(required):
            ok = self.verify_step(provided_steps[i], required[i])
            if not ok:
                self.add_log("PROCESS_STEPS_FAILED step_mismatch tid=" + str(tid) +
                             " index=" + str(i))
                return False
            i = i + 1
        return self.complete_task(uid, tid)

    def add_log(self, message):
        self.store["logs"].append(message)

    def show_last_logs(self, uid, count):
        logs = self.store["logs"]
        filtered = []
        i = 0
        while i < len(logs):
            if ("uid=" + str(uid)) in logs[i]:
                filtered.append(logs[i])
            i = i + 1
        start = len(filtered) - count
        if start < 0:
            start = 0
        idx = start
        while idx < len(filtered):
            print(filtered[idx])
            idx = idx + 1

    def search_logs_by_keyword(self, keyword):
        logs = self.store["logs"]
        i = 0
        while i < len(logs):
            if keyword in logs[i]:
                print(logs[i])
            i = i + 1

def count_words(text):
    word = ""
    words = []
    i = 0
    while i < len(text):
        c = text[i]
        if c == " " or c == "," or c == "." or c == "!" or c == "?":
            if word != "":
                words.append(word)
                word = ""
        else:
            word = word + c
        i = i + 1
    if word != "":
        words.append(word)
    return len(words)

def find_longest_word(text):
    longest = ""
    word = ""
    i = 0
    while i < len(text):
        c = text[i]
        if c == " " or c == "," or c == "." or c == "!" or c == "?":
            if len(word) > len(longest):
                longest = word
            word = ""
        else:
            word = word + c
        i = i + 1
    if len(word) > len(longest):
        longest = word
    return longest

def encrypt_text(text, shift):
    result = ""
    i = 0
    while i < len(text):
        c = text[i]
        code = ord(c)
        if c >= "a" and c <= "z":
            base = ord("a")
            offset = code - base
            offset = (offset + shift) % 26
            result = result + chr(base + offset)
        elif c >= "A" and c <= "Z":
            base = ord("A")
            offset = code - base
            offset = (offset + shift) % 26
            result = result + chr(base + offset)
        else:
            result = result + c
        i = i + 1
    return result

def decrypt_text(text, shift):
    return encrypt_text(text, -shift)

def recursive_sum(n):
    if n <= 0:
        return 0
    return n + recursive_sum(n - 1)

def recursive_reverse(text):
    if text == "":
        return ""
    return recursive_reverse(text[1:]) + text[0]

def simulate_points(uid, rounds, lab):
    total = 0
    r = 0
    while r < rounds:
        base = (r % 5) + (uid % 3)
        if (uid + 2) % 2 == 0:
            gained = base + 2   
        else:
            gained = base - 1  
        total = total + gained
        lab.update_score(uid, gained)
        r = r + 1
    return total

lab = CyberLab(data_store)

def add_user(uid, name):
    return lab.add_user(uid, name)

def find_user(uid):
    return lab.find_user(uid)

def update_score(uid, value):
    return lab.update_score(uid, value)

def add_log(message):
    return lab.add_log(message)

def create_task(tid, title, steps):
    category = "general"
    if "network" in title.lower():
        category = "network"
    elif "crypto" in title.lower():
        category = "crypto"
    elif "web" in title.lower():
        category = "web"
    return lab.create_task(tid, title, steps, difficulty=2, category=category)

def get_task(tid):
    return lab.get_task(tid)

def complete_task(uid, tid):
    return lab.complete_task(uid, tid)

def verify_step(step_text, requirement):
    return lab.verify_step(step_text, requirement)

def process_steps(uid, tid, provided_steps):
    return lab.process_steps(uid, tid, provided_steps)

def summarize_user(uid):
    return lab.summarize_user(uid)

def search_users_by_letter(letter):
    return lab.search_users_by_letter(letter)

def sort_users_by_score():
    return lab.sort_users_by_score()

def simulate_points_wrapper(uid, rounds):
    return simulate_points(uid, rounds, lab)

def initialize_sample_data():
    add_user(1, "Alice")
    add_user(2, "Bob")
    add_user(3, "Charlie")
    add_user(4, "Dave")

    create_task(
        101,
        "Network scanning basics",
        ["nmap", "22", "open"]
    )

    create_task(
        102,
        "Web login brute-force simulation",
        ["login", "password", "lockout"]
    )

    create_task(
        103,
        "Crypto message decryption",
        ["cipher", "key", "plain"]
    )

    add_log("SYSTEM_INIT Cyber Security Lab ready.")

def print_title(text):
    print()
    print("=" * 60)
    print(text)
    print("=" * 60)

def print_subtitle(text):
    print()
    print("-" * 40)
    print(text)
    print("-" * 40)

def run():
    initialize_sample_data()
    print_title("CYBER SECURITY LAB - UNIVERSITY ASSIGNMENT SIMULATOR")
    print_subtitle("1. Simulate task submissions (process_steps + complete_task)")
    uid = 1
    tid = 101
    provided = ["nmap scan all ports", "22/tcp", "open state confirmed"]
    success = process_steps(uid, tid, provided)
    
    print("User", uid, "submitting task", tid, "success:", success)

    uid = 2
    tid = 102
    provided = ["login attempt", "wrong keyword", "account locked"]
    success = process_steps(uid, tid, provided)
    
    print("User", uid, "submitting task", tid, "success:", success)

    print_subtitle("2. Simulate extra practice rounds (simulate_points)")
    total_points = simulate_points_wrapper(1, 5)
    
    print("User 1 gained extra points from practice rounds:", total_points)

    print_subtitle("3. Text analysis of student report (count_words, find_longest_word)")
    report = "This lab demonstrates basic network scanning and simple brute force defense."
    wcount = count_words(report)
    longest = find_longest_word(report)
    
    print("Report text:", report)
    print("Word count =", wcount)
    print("Longest word =", longest)

    print_subtitle("4. Encrypted assignment hint (encrypt_text / decrypt_text)")
    hint = "Check the open SSH port before attempting brute force."
    enc = encrypt_text(hint, 5)
    dec = decrypt_text(enc, 5)
    
    print("Original hint:", hint)
    print("Encrypted hint:", enc)
    print("Decrypted hint:", dec)

    print_subtitle("5. Reverse obfuscated malware string (recursive_reverse)")
    malware_string = "edoctam_erawlaM"
    restored = recursive_reverse(malware_string)
    
    print("Obfuscated string:", malware_string)
    print("Restored string:", restored)

    print_subtitle("6. Recursive sum as scoring reference (recursive_sum)")
    n = 10
    s = recursive_sum(n)
    print("recursive_sum(", n, ") =", s)

    print_subtitle("7. Search students by letter (search_users_by_letter)")
    found = search_users_by_letter("a")  
    i = 0
    while i < len(found):
        u = found[i]
        print("Found user:", u.uid, u.name, "score:", u.score)
        i = i + 1

    print_subtitle("8. User summaries (summarize_user)")
    i = 0
    while i < len(data_store["users"]):
        u = data_store["users"][i]
        info = summarize_user(u.uid)
        print("UID:", info["uid"],
              "Name:", info["name"],
              "Level:", info["level"],
              "Score:", info["score"],
              "AvgDelta:", info["average"],
              "Tasks:", info["completed_tasks"])
        i = i + 1

    print_subtitle("9. Ranking by score (sort_users_by_score)")
    sorted_users = sort_users_by_score()
    rank = 1
    i = 0
    while i < len(sorted_users):
        u = sorted_users[i]
        print("Rank", rank, "-", u.uid, u.name, "Score:", u.score)
        rank = rank + 1
        i = i + 1

    print_subtitle("10. Show last logs for Alice (show_last_logs) and keyword search")
    lab.show_last_logs(1, 5)
    print()
    print("Logs containing 'PROCESS_STEPS_FAILED':")
    lab.search_logs_by_keyword("PROCESS_STEPS_FAILED")

    print_subtitle("END OF SIMULATION")

if __name__ == "__main__":
    run()
